In [ ]:
# !pip install -e /files/shared/ap/jupyter-notebook/pkgs/objFuncs 
# !pip install -e /files/shared/ap/jupyter-notebook/pkgs/pyBO

 # please re-start kernel if these packages are installed for the first time

In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import time
import datetime
import concurrent
from phantasy import caget, fetch_data

In [ ]:
import objFuncs
from objFuncs.util import read_BPMoverview_snapshot
from objFuncs import maximize_FC
from objFuncs.util import plot_obj_history
from pyBO import pyBO

In [ ]:
budget = 20
n_init = 10

# check objFuncs machineIO, source and beam  

In [ ]:
objFuncs._global_machineIO._test = False
objFuncs._global_machineIO._fetch_data_time_span = 5.05
objFuncs._global_machineIO._ensure_set_timewait_after_ramp = 0.5
# objFuncs._global_machineIO.view()

In [ ]:
SCS = caget("ACS_DIAG:DEST:ACTIVE_ION_SOURCE")
ion = caget("FE_ISRC"+str(SCS)+":BEAM:ELMT_BOOK")
Q = caget("FE_ISRC"+str(SCS)+":BEAM:Q_BOOK")
A = caget("FE_ISRC"+str(SCS)+":BEAM:A_BOOK")
# AQ = caget("FE_ISRC2:BEAM:MOVRQ_BOOK")
AQ = A/Q
ion = str(A)+ion+str(Q)
print('SCS'+str(SCS), ion, 'A/Q=',AQ)

In [ ]:
now0 = datetime.datetime.now()
now0str = now0.strftime('%Y%m%d_%H%M')
fname = now0str+'['+ion+'][pyBO][LS1][CA]match_phase_to_snapshot'
fname

# preprare objective

In [ ]:
decision_CSETs=['FE_MEBT:RFC_D1066:PHA_CSET'   , 'FE_MEBT:RFC_D1107:PHA_CSET'   ]
decision_RDs = ['FE_MEBT:RFC_D1066:PHA_RD_CAVS', 'FE_MEBT:RFC_D1107:PHA_RD_CAVS']
decision_tols = [0.05, 0.05]
ave,_ = fetch_data(decision_CSETs,0.1)

decision_min = ave - 2
decision_max = ave + 2

print(ave)
print(decision_min)
print(decision_max)

In [ ]:
# BPM_snapshot_fname = '20230730_0235_48Ca10_2cs_216p7MeVu_100W_pulsed_BDSBD.bpm'
BPM_snapshot_fname = '20230728_1635_36Ar10_210MeVu_10p2kW.bpm'
objective_goal = read_BPMoverview_snapshot(BPM_snapshot_fname,Dnum_from=1100,Dnum_to=1222)
objective_goal

In [ ]:
objective_norm = {k:1. for k in objective_goal.keys()}
objective_weight = {k:1. for k in objective_goal.keys()}

In [ ]:
obj = objFuncs.objFuncGoals(
    decision_CSETs=decision_CSETs,
    decision_RDs = decision_RDs,
    decision_min = decision_min,
    decision_max = decision_max,
    decision_tols = decision_tols,
    objective_goal = objective_goal,
    objective_weight = objective_weight,
    objective_norm = objective_norm,
    apply_bilog = True,
)

 # Prepare plot callbacks

In [ ]:
# define what to plot
plot_CSETs = plot_obj_history(
                obj.history['decision_CSETs'],
                title = 'decision_CSETs',
                inline = True,
                )
plot_RDs = plot_obj_history(
                obj.history['objective_RDs'],
                title = 'objective_RDs',
                inline = True,
                )
plot_objs = plot_obj_history(
            obj.history['objectives'],
            title = 'objectives',
            inline = True,
            add_y_data = obj.history['objectives']['total'],
            add_y_label = 'total obj'
            )
callbacks = [plot_CSETs,plot_RDs,plot_objs]

# evaluate objective and plot
def obj_callbacks(x):
    return obj(x,callbacks=callbacks)

# Run BO

### run BO until budget exhaust

In [ ]:
# run optimizer
bo, X_pending, Y_pending_future = pyBO.runBO(
                                    obj_callbacks,  
                                    bounds=obj.decision_bounds,
                                    n_init=n_init,
                                    x0 = obj.x0,
                                    budget = budget,
                                    batch_size=1,
#                                     timeout = 2+objFuncs._global_machineIO._fetch_data_time_span,
                                    path="./log/",
                                    tag=fname+'_pyBO_history',
                                    write_log = False)


# exploit model for a few iteration
acquisition_func_args = {'beta':0.01}
X_pending, Y_pending_future= bo.loop( 
                                n_loop=1,  # number of additional optimization interation
                                func_obj = obj_callbacks,
                                bounds = obj.decision_bounds,
                                acquisition_func_args = acquisition_func_args,
                                X_pending = X_pending, 
                                Y_pending_future = Y_pending_future,
                                batch_size = 1,
                                write_log = False,
                                )

for f in callbacks:
    f.close()

### Fine Tune near current best

In [ ]:
# local optimization near the best_solution
bounds_diff = obj.decision_bounds[:,1] - obj.decision_bounds[:,0]
for i in range(3):
    x_best,y_best = bo.best_sofar()
    bounds_best = np.array(list(zip(x_best-0.08*bounds_diff, x_best+0.08*bounds_diff)))
    acquisition_func_args = {'beta':1}

    X_pending, Y_pending_future= bo.loop( 
                                    n_loop=2,  # number of additional optimization interation
                                    func_obj = obj_callbacks,
                                    bounds = bounds_best,
                                    acquisition_func_args = acquisition_func_args,
                                    X_pending = X_pending, 
                                    Y_pending_future = Y_pending_future,
                                    batch_size = 1,
                                    write_log = False,
                                    )
for f in callbacks:
    f.close()

In [ ]:
ax = bo.plot_obj_history()
ax.vlines(n_init,*ax.get_ylim(),color='k')

### set to best solution 

In [ ]:
x_best,y_best_old = bo.best_sofar()
y_best_new = obj(x_best)
print(x_best,y_best_old[0],y_best_new)   # check if best solution objective value is consistent

In [ ]:
obj.write_log(fname=os.path.join('./log',fname))

In [ ]:
now1 = datetime.datetime.now()
now1str = now1.strftime('%Y%m%d_%H%M')
time_delta = now1 - now0
print("time took:",str(time_delta.total_seconds()/60)[:4],'min')

# Visualize Surrogate model

In [ ]:
# fixed_values_for_each_dim = {2:x_best[2],3:x_best[3]}  # fix values to visualize high dim surrogate model
fixed_values_for_each_dim = None                         # do not fix values but project maximum. Can take long time to plot
batch_size = 1

In [ ]:
# plot surrogate mean model of the last epoch. Can take long time for decision dim >= 4

t0 = time.monotonic()
from math import ceil
nplot = int(0.5*len(obj.decision_CSETs))
nrow = ceil(0.5*nplot)
fig,ax = plt.subplots(nrow,2,figsize=(8,3*nrow))
for i in range(nrow):
    for j in range(2):
        n = 2*i+j
        if nrow>1:
            ax_ = ax[i,j]
        else:
            ax_ = ax[j]
        if n >= nplot:
            ax_.set_visible(False)
            break
        bo.plot_model_2D_projection(project_maximum=True,
                                    dim_xaxis = 2*n,
                                    dim_yaxis = 2*n+1,
                                    grid_ponits_each_dim = 16,
                                    fixed_values_for_each_dim=fixed_values_for_each_dim,
                                            fig=fig,ax=ax_);
        ax_.scatter(obj.x0[2*n],obj.x0[2*n+1],color='k',label='initial')
        ax_.scatter(x_best[2*n],x_best[2*n+1],color='r',label='optimum')
        ax_.set_xlabel(obj.decision_CSETs[2*n  ])
        ax_.set_ylabel(obj.decision_CSETs[2*n+1])
        ax_.legend()
fig.tight_layout()

    
print('time took: ',time.monotonic()-t0)